In [1]:
from model import DataLoader, FineTunedModel

c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['c:\\Users\\user\\Desktop\\WN 22\\EECS 487\\eecs487-finalproject\\model', 'c:\\Python310\\python310.zip', 'c:\\Python310\\DLLs', 'c:\\Python310\\lib', 'c:\\Python310', '', 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python310\\site-packages', 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python310\\site-packages\\win32', 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python310\\site-packages\\win32\\lib', 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python310\\site-packages\\Pythonwin', 'c:\\Python310\\lib\\site-packages', '..']


In [2]:

data = DataLoader()



Currently processing row 0
Currently processing row 1
Currently processing row 2
Currently processing row 3
Currently processing row 4
Currently processing row 5
Currently processing row 6
Currently processing row 7
Currently processing row 8
Currently processing row 9
Currently processing row 10
Currently processing row 11
Currently processing row 12
Currently processing row 13
Currently processing row 14
Currently processing row 15
Currently processing row 16
Currently processing row 17
Currently processing row 18
Currently processing row 19
Currently processing row 20
Currently processing row 21
Currently processing row 22
Currently processing row 23
Currently processing row 24
Currently processing row 25
Currently processing row 26
Currently processing row 27
Currently processing row 28
Currently processing row 29
Currently processing row 30
Currently processing row 31
Currently processing row 32
Currently processing row 33
Currently processing row 34
Currently processing row 35
Cu

In [9]:
# split the data into train, test, and validation 
from sklearn.model_selection import train_test_split

# split train as 50%, val as 25%, test as 25% 
X_train, X_test = train_test_split(
    data.data, test_size = 0.25, random_state=487, shuffle=True 
)

X_train, X_val = train_test_split(
    X_train, test_size = 0.33, random_state=487, shuffle=True
)



In [8]:
from model import *
import torch
from sklearn.neighbors import NearestNeighbors 
from torch.optim import Adam
from torch.nn import CosineEmbeddingLoss
import numpy as np
lr = 0.001
model = FineTunedModel()
optim = Adam(model.parameters(), lr=lr)
criterion = CosineEmbeddingLoss() # cosine similarity 
# start the training process 
num_epoch = 30

# TODO: do the actual data loading 

def performance(data_partition, k = 10):
    
    mrr = 0
    acc = 0
    num_samples = 0
    for entry in data_partition:
        # if the label is 0, we don't need to check 
        if entry["label"] == -1:
            continue
        num_samples += 1
        # first get the new embedding using our model
        # We use Q2 because Q2 is the duplicate 
        cur_embed = model.ff(entry["Q2_title"])
        # we also need to get the real id 
        original_label = entry["Q1_id"]
        
        nearest_neighbors = knn.kneighbors(cur_embed.detach().reshape(1, -1), k, return_distance=False).tolist()[0]
        
        # we need to start at the second index, because the first index is always gonan be itself 
        
        # acc
        if data.titles_id[nearest_neighbors[1]] == original_label: 
            acc += 1
            
        # mrr
        for i, cur_neighbor in enumerate(nearest_neighbors[1:]):
            
            if data.titles_id[cur_neighbor] == original_label:
                mrr += 1 / (1 + i)
                break
    
    mrr /= num_samples
    acc /= num_samples
    return acc, mrr
for epoch in range(num_epoch):
    
    # before we start our loop, we need to retrain our nearest neighbors
    cur_samples =[] 
    
    with torch.no_grad():
        for unique_title in data.unique_titles:
            cur_samples.append(np.array(model.ff(unique_title)))
    # print("Cur sample first index", cur_samples[0])
    knn = NearestNeighbors(n_neighbors=2, metric="cosine", n_jobs=-1)
    knn.fit(cur_samples)
    
    # now calcualte the mrr and accuracy
    vacc, vmrr = performance(X_val)
    tacc, tmrr = performance(X_train)
    print("Epoch:", epoch+1, " | Training Accuracy", tacc, " | Training MRR:", tmrr)
    print("Epoch:", epoch+1, " | Validation Accuracy", vacc, " | Validation MRR:", vmrr)
    
    # now start training process 
    total_loss = 0
    for entry in X_train:
        q1 = entry["Q1_title"]
        q2 = entry["Q2_title"]
        y = entry["label"]
        
        optim.zero_grad()   
        # this output a "new embedding" for both q1 and q2 
        out1, out2, _ = model(q1, q2)
        
        loss = criterion(out1, out2, torch.tensor(y, dtype=torch.float))
        loss.backward()
        optim.step()
        total_loss += loss.item()
        
    print("Epoch:", epoch+1, "with a loss of:", total_loss/len(data))
    print("\n")
        

Epoch: 1  | Training Accuracy 0.0  | Training MRR: 0.004166666666666667
Epoch: 1  | Validation Accuracy 0.0  | Validation MRR: 0.010869565217391304
Epoch: 1 with a loss of: 0.00018371701240539552


Epoch: 2  | Training Accuracy 0.0  | Training MRR: 0.007407407407407407
Epoch: 2  | Validation Accuracy 0.043478260869565216  | Validation MRR: 0.05512422360248447
Epoch: 2 with a loss of: 1.4371275901794434e-05


Epoch: 3  | Training Accuracy 0.0  | Training MRR: 0.01619708994708995
Epoch: 3  | Validation Accuracy 0.043478260869565216  | Validation MRR: 0.04891304347826087
Epoch: 3 with a loss of: 7.252991199493408e-06


Epoch: 4  | Training Accuracy 0.0  | Training MRR: 0.014351851851851853
Epoch: 4  | Validation Accuracy 0.0  | Validation MRR: 0.032781228433402344
Epoch: 4 with a loss of: 4.475712776184082e-06


Epoch: 5  | Training Accuracy 0.0  | Training MRR: 0.017916666666666668
Epoch: 5  | Validation Accuracy 0.0  | Validation MRR: 0.02795031055900621
Epoch: 5 with a loss of: 3.09616